In [1]:
#data loading
import pandas as pd

predata = pd.read_csv('C:/Users/BeatrizCarvalho/OneDrive - Closer Consultoria Lda/Documents/Entangled-Spaces/Payments/Datasets/0payments_nd_corr.csv', low_memory = False)

#create column weekday
predata['weekday'] = pd.to_datetime(predata['timestamp']).apply(lambda x: x.weekday())

#create column month
predata['month'] = pd.DatetimeIndex(predata['timestamp']).month

#drop column timestamp that was used for creating weekday and month columns
predata.drop(['timestamp'], axis = 1, inplace = True)


#renaming browsers
def assign_brws_fam(x):
    changes = {
        "m bot": "other",  #15
        "android": "other",  #104
        "1password": "other",  #57
        "chrome mobile": "chrome",  #735168
        "chrome mobile webview": "chrome",  #2681
        "mobile safari": "safari",  #338413
        "mobile safari ui/wkwebview": "safari",  #4190964
        "firefox mobile": "firefox",  #5515
        "edge mobile": "edge",
        "firefox mobile": "firefox"
    }

    if x in changes.keys():
        return changes[x]
    else:
        return x


predata["browser_family"] = predata["browser_family"].apply(lambda x: assign_brws_fam(x))


#renaming os
def assign_os_fam(x):
    changes = {
        "windows phone": "windows"  #863
    }

    if x in changes.keys():
        return changes[x]
    else:
        return x


predata["os_family"] = predata["os_family"].apply(lambda x: assign_os_fam(x))


#remove spaces on browsers and os
cols = ['os_family', 'browser_family']

for col in cols:
    predata[col] = predata[col].apply(lambda x: x.replace(' ', ''))


#make all lowercase
def convert_column_lower(column, df):
    df[column] = df[column].str.lower()


columns_to_lower = ['canal', 'operativa', 'browser_family', 'os_family', 'ipaddress', 'trusted_indicator']
for col in columns_to_lower:
    predata[col] = predata[col].str.lower()

    
#remove dots from ipaddress
predata['ipaddress'] = predata['ipaddress'].apply(lambda x: "".join(x.split(".")))


#merge is_tablet, is_pc, is_mobile, is_touch on a new column called device
def deviceselect(x):
    #if True in tablet and touch
    if x['is_tablet'] and x['is_touch']:
        return 'ttablet'

    #if True in pc and touch
    if x['is_pc'] and x['is_touch']:
        return 'tpc'

    #if True in mobile and touch
    if x['is_mobile'] and x['is_touch']:
        return 'tmobile'

    #if True only in mobile
    if x['is_mobile']:
        return 'mobile'

    #if True only in pc
    if x['is_pc']:
        return 'pc'

    #if True only in tablet
    if x['is_tablet']:
        return 'tablet'

    #if none of the above return otherd
    return 'otherd'

predata['device'] = predata.apply(deviceselect, axis = 1)


#drop columns that were merged on the new device column
predata.drop(['is_pc', 'is_tablet', 'is_mobile', 'is_touch'], axis = 1, inplace = True)

#change data types for columns that need to be categorical
convert_dict = {'clientid': object, 'entity': object, 'reference': object, 'iban_orig': object, 'iban_dest': object,
                'hour': object, 'week': object, 'weekday': object, 'month': object}

predata = predata.astype(convert_dict)

In [2]:
#create new dataframe with already existent data but splitted in labels 
frauds = predata[predata['is_fraud'] == 1].copy(deep = True)

#choose sample
non_frauds = predata[predata['is_fraud'] == 0].sample(5000).copy(deep = True)

del predata

#new dataframe concatenated
result = pd.concat([non_frauds, frauds], ignore_index = True).copy(deep = True)

del frauds, non_frauds

In [3]:
#dummies for categorical columns
dums = pd.get_dummies(result, columns = ['canal', 'operativa', 'trusted_indicator', 'browser_family', 'os_family', 'hour', 
                                         'week', 'weekday', 'month', 'device'])

In [4]:
#assign target column
y = dums['is_fraud']

X = dums.drop('is_fraud', axis = 1)

In [5]:
#ignore errors in conversion to numerics and do not allow the conversion
X = X.apply(pd.to_numeric, errors = 'ignore')

In [6]:
#adjust mixed data in ips - unknown and numeric
def adjust_ip(ip):
    try:
        return int(ip)
    except:
        return 0


X['ipaddress'] = X['ipaddress'].apply(lambda x: adjust_ip(x))

In [7]:
#apply to the target column
y = y.apply(pd.to_numeric)

In [8]:
#smote
from imblearn.over_sampling import SMOTE
import multiprocessing

over = SMOTE()

X_res, y_res = over.fit_resample(X, y)

In [9]:
#copy to new dataframe
X_res['is_fraud'] = y_res.copy()

C:\Users\BEATRI~1\AppData\Local\Temp/ipykernel_14836/3657357880.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_res['is_fraud'] = y_res.copy()


In [10]:
#new dataframe shape
X_res.shape

(10000, 131)

In [11]:
#when smote does not attribute any value choose one accordingly to the mode
dummies_cols = [('canal', 'mbp'), ('operativa', 'pagsrv'), ('trusted_indicator', '0'), ('browser_family', 'chrome'), 
                ('os_family', 'windows'), ('hour', '11'), ('week', '21'), ('weekday', '0'), ('month', '5'), ('device', 'pc')]

In [12]:
#undo dummies
def get_proper_value(df, created_cols, origin_col, default_val):
    proper_result = []

    for _, row in df.iterrows():
        curr_values = row[created_cols]
        was_set = False
        for i, val in enumerate(curr_values.values):
            if val == 1:
                was_set = True
                to_append = str(created_cols[i])
                to_append = to_append.replace(origin_col, '')
                proper_result.append(to_append)
                break
        if not was_set:
            proper_result.append(default_val)
    return proper_result

In [13]:
#assign the reconverted values
for (dummie, default) in dummies_cols:
    curr_dummie = dummie + '_'
    used_cols = [curr for curr in X_res.columns if curr_dummie in curr]
    X_res[dummie] = get_proper_value(X_res, used_cols, curr_dummie, default)

In [14]:
#remove dummy columns
remove_cols = ['canal_', 'operativa_', 'trusted_indicator_', 'browser_family_', 'os_family_', 'hour_', 'week_', 'weekday_',
               'month_', 'device_']

In [15]:
for rem in remove_cols:
    used_cols = [curr for curr in X_res.columns if rem in curr]
    for c in used_cols:
        X_res.drop(c, inplace = True, axis = 1)

In [16]:
#confirm the final columns
X_res.columns

Index(['entity', 'reference', 'iban_orig', 'iban_dest', 'amount',
       'accountbalance', 'ipaddress', 'clientid', 'is_fraud', 'canal',
       'operativa', 'trusted_indicator', 'browser_family', 'os_family', 'hour',
       'week', 'weekday', 'month', 'device'],
      dtype='object')

In [17]:
#assign the value unknown to the ip first assigned as zero
def adjust_ip_reverse(ip):
    if ip == 0:
        return 'unknown'
    return ip

X_res['ipaddress'] = X_res['ipaddress'].apply(lambda x: adjust_ip_reverse(x))

In [18]:
#final distribution ratio
X_res['is_fraud'].value_counts() / X_res.shape[0]

0    0.5
1    0.5
Name: is_fraud, dtype: float64

In [19]:
#save csv
X_res.to_csv('C:/Users/BeatrizCarvalho/OneDrive - Closer Consultoria Lda/Documents/Entangled-Spaces/Payments/Datasets/2payments_balanced_smote+rund-10m.csv', index = False)